In [1]:
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import urllib.request 
import urllib.parse
import urllib.error
import ssl
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import time

# For ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [2]:
#price
def get_general_data():
    l_price=[]
    bed=[]
    baths=[]
    status_type=[]
    status_text=[]
    zipcode=[]
    house_link=[]
    address=[]
    city=[]
    sqrft=[]
    price_sqft=[]
    price_change=[]
    buildername=[]
    price_reduction=[]
    data=result['cat1']['searchResults']['listResults'] #the necessary data is in a nested dictionary
    for house in data:
        l_price.append(house['price'])
        bed.append(house['beds'])
        baths.append(house['baths'])
        status_type.append(house['statusType'])
        status_text.append(house['statusText'])
        zipcode.append(house['addressZipcode'])
        house_link.append(house['detailUrl'])
        address.append(house['address'])
        city.append(house['hdpData']['homeInfo']['city'])
        sqrft.append(house['area'])
    df['List Price']=l_price
    df['No of Beds']=bed
    df['No of bath']=baths
    df['Status_type']=status_type
    df['Status Text']=status_text
    df['Zipcode']=zipcode
    df['Address']=address
    df['City']=city
    df['Sqrft']=sqrft
    df['url']=house_link
    
    print('list price',len(l_price))
    print('# bed',len(bed))
    print('#bath',len(baths))
    print('Status_type',len(status_type))
    print('Status Text',len(status_text))
    print('Zipcode',len(zipcode))
    print('address',len(address))
    print('city',len(city))
    print('sqrft',len(sqrft))
    print('url',len(house_link))
    return df


In [3]:
#get Price/sqft for each house

def get_house_specific_data():
    import time
    time.sleep(0.5)
    house_types=[]
    year_built=[]
    price_per_sf=[]
    HOA=[]
    E_rating=[]
    J_rating=[]
    H_rating=[]
    time_zillow=[]
    views=[]
    saves=[]

    urls=df['url']

    for j in urls:
        
        req1 = Request(j, headers={'User-Agent': 'Mozilla/5.0'})
        webpage1 = urlopen(req1)
        soup1=BS(webpage1,'html.parser')

        home_facts=soup1.findAll('li',attrs={'class':'ds-home-fact-list-item'})

        house_type=re.findall('^.*:(.*)$',home_facts[0].get_text())[0]
        yr_built=re.findall('^.*:(.*)$',home_facts[1].get_text())[0]
        
        home_details=''
        for i in home_facts:
            home_details+=i.get_text()
            
        try:
            ps=re.findall('^.*Price/sqft:(\$[0-9,]{1,6}).*$',home_details)[0]
            price_per_sf.append(ps)
        except:
            price_per_sf.append('NA')
        
        try:
            hoa=re.findall('^.*HOA:(\$[0-9,]{1,5}).*$',home_details)[0]
            HOA.append(hoa)
        except:
            HOA.append('NA')
        
        house_types.append(house_type)
        year_built.append(yr_built)    
        
        #school rating
        content=soup1.findAll('div',attrs={'class':'Spacer-c11n-8-48-0__sc-17suqs2-0 bcAgQH'})
        str1=''
        for i in content:
            str1+=i.get_text()
        rating=re.findall('([0-9]{1,2})/10',str1)
        #print(rating)
        try:
            if len(rating)==3:
                E_rating.append(rating[0])
                J_rating.append(rating[1])
                H_rating.append(rating[2])
            elif len(rating)==2:
                if len(re.findall('(Elementary|K-5|K-6)',str1))==0:
                    E_rating.append('NA')
                    J_rating.append(rating[0])
                    H_rating.append(rating[1])
                elif len(re.findall('(Middle|Junior High|Junior|7-8)',str1))==0:
                    E_rating.append(rating[0])
                    H_rating.append(rating[1])
                    J_rating.append('NA')
                elif len(re.findall('(Junior High)',str1))==0 and len(re.findall('(High|9-12)',str1))==1:
                    E_rating.append(rating[0])
                    J_rating.append(rating[1])
                    H_rating.append('NA')
                else:
                    E_rating.append('NA')
                    J_rating.append('NA')
                    H_rating.append('NA')
                    
            elif len(rating)==1:
                if len(re.findall('(Elementary|K-5|K-6)',str1))==1:
                    E_rating.append(rating[0])
                    J_rating.append('NA')
                    H_rating.append('NA')
                elif len(re.findall('(Middle|Junior High||Junior|7-8)',str1))==1:
                    J_rating.append(rating[0])
                    E_rating.append('NA')
                    H_rating.append('NA')
                elif len(re.findall('(High|9-12)',str1))==1:
                    H_rating.append(rating[0])
                    E_rating.append('NA')
                    J_rating.append('NA')
                else:
                    E_rating.append('NA')
                    J_rating.append('NA')
                    H_rating.append('NA')
                        
            else:
                E_rating.append('NA')
                J_rating.append('NA')
                H_rating.append('NA')
        except:
            E_rating.append('NA')
            J_rating.append('NA')
            H_rating.append('NA')
        
            
        #time,saves & views
        
        
        count=soup1.findAll('div',attrs={'class':"ds-expandable-card-section-default-padding"})
        tsv=''
        for i in count:
            tsv+=i.get_text()
        try:
            time=re.findall('Time on Zillow([0-9,]+) days',tsv)[0]
            time_zillow.append(time)
        except:
            time_zillow.append('NA')
        try:
            view=re.findall('Views([0-9,]+)',tsv)[0]
            views.append(view)
        except:
            views.append('NA')
        try:
            save=re.findall('Saves([0-9,]+)',tsv)[0]
            saves.append(save)
        except:
            saves.append('NA')
        

        
    df['House_type']=house_types
    df['Year Built']=year_built
    df['Price/sqft']=price_per_sf
    df['Time on Zillow']=time_zillow
    df['Views']=views
    df['Saves']=saves
    df[ 'Elementary school rating']=E_rating
    df['Junior school rating']=J_rating
    df['High school rating']=H_rating
    df['HOA Fee']=HOA
        
    print('House_type:',len(house_types))
    print('Year Built',len(year_built))
    print('price/sqrtft',len(price_per_sf))
    print('HOA Fee',len(HOA))
        
    print('E_rating length:',len(E_rating))
    print('J_rating length:',len(J_rating))
    print('H_rating length:',len(H_rating))
    print('saves length',len(saves))
    print('views lenth',len(views))
    print('Time on zillow length',len(time_zillow))
    return df

In [4]:
def to_dataframe():
    data=pd.DataFrame(df)
    writer=pd.ExcelWriter('zillow_page{}.xlsx'.format(i))
    data.to_excel(writer)
    writer.save()
    writer.close()

# How to run?

In [6]:
link=input('Enter the url:')
sp=int(input('start page'))
ep=int(input('End page'))
for i in range(sp,ep+1):
    time.sleep(2)
    url=str(link)+str(i)+'_p/'
    print(url)
    req=Request(url, headers={'User-Agent':'Mozilla/5.0'})
    webpage=urlopen(req).read()
    soup=BS(webpage, 'html.parser')
    house_info=soup.find(text=re.compile('<!--(.+?)-->')) #locate data from comments; there are more comments but they aren't important, so I use .find, not .findAll
    house_info
    house_info=house_info[4:-3] #get rid of the comment indicators
    result=json.loads(house_info)
    result
    
    df=dict()
    get_general_data()
    get_house_specific_data()
    to_dataframe()

Enter the url:https://www.zillow.com/fremont-ca/
start page1
End page2
https://www.zillow.com/fremont-ca/1_p/
list price 40
# bed 40
#bath 40
Status_type 40
Status Text 40
Zipcode 40
address 40
city 40
sqrft 40
url 40


IndexError: list index out of range